In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import commonutils
import models

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance

from dataclasses import dataclass
import prettyprinter as pp

from sklearn.cross_decomposition import PLSRegression
import warnings
import sys

from sklearn import preprocessing

from copy import deepcopy
import pickle

In [2]:
warnings.simplefilter("ignore")

howmanydifs = 3
allvalues_perset = pickle.load(open("./data/allvalues_perset.p", "rb"))
methods = pickle.load(open("./data/methods.p", "rb"))
fullsetnames = pickle.load(open("./data/fullsetnames.p", "rb"))
functionals = pickle.load(open("./data/functionals.p", "rb"))
basis_sets = pickle.load(open("./data/basis_sets.p", "rb"))
supersetnames = pickle.load(open("./data/supersetnames.p", "rb"))

In [ ]:
from importlib import reload
reload(commonutils)

from commonutils import ModelResults

allfeatures = set()
for setname in fullsetnames:
    for val in allvalues_perset[setname]:
        for k in val:
            if k.find("energydiff") != -1:
                for f in val[k]:
                    allfeatures.add(f)

# set labels and sets iists
models_results = {}
for setname in fullsetnames:
    models_results[setname] = ModelResults()
    for val in allvalues_perset[setname]:
        models_results[setname].labels.append(val["label"]) 
        models_results[setname].supersetnames.append(val["super_setname"])
        models_results[setname].setnames.append(val["super_setname"]+"_"+val["setname"])

insidemethods = ["W","D3(0)","D3(BJ)"]
for setname in fullsetnames:
    for methodid in range(howmanydifs):
        y_pred = []
        for val in allvalues_perset[setname]:
            y_pred.append(val["label"] + val["difs"][methodid])

        wtmad = None
        fulllist = list(supersetnames.keys()) + ["Full"]
        if setname in fulllist:
            wtmadf = commonutils.wtmad2(models_results[setname].setnames, \
                                    models_results[setname].labels, y_pred)
            wtmad = wtmadf[setname]

            if wtmad < models_results[setname].bestinsidemethod_wtmad:
                models_results[setname].bestinsidemethod_wtmad = wtmad
                models_results[setname].bestinsidemethod_wtmad_name = insidemethods[methodid]
                models_results[setname].y_pred_bestinsidemethod_wtmad = y_pred

        rmse = mean_squared_error(models_results[setname].labels, \
                                y_pred, squared=False)

        if rmse < models_results[setname].bestinsidemethod_rmse:
            models_results[setname].bestinsidemethod_rmse = rmse
            models_results[setname].bestinsidemethod_rmse_name = insidemethods[methodid]
            models_results[setname].y_pred_bestinsidemethod_rmse = y_pred

    for j, method in enumerate(methods):
        y_pred = []
        for val in allvalues_perset[setname]:
            y_pred.append(val[method + "_energydiff"][method+"_FINAL_SINGLE_POINT_ENERGY"])

        wtmad = None            
        fulllist = list(supersetnames.keys()) + ["Full"]
        if setname in fulllist:
            wtmadf = commonutils.wtmad2(models_results[setname].setnames, \
                                models_results[setname].labels, y_pred)
            wtmad = wtmadf[setname]

            if wtmad < models_results[setname].bestourmethod_wtmad:
                models_results[setname].bestourmethod_wtmad = wtmad
                models_results[setname].bestourmethod_wtmad_name = method
                models_results[setname].y_pred_bestourmethod_wtmad = y_pred
        
        rmse = mean_squared_error(models_results[setname].labels,\
                                y_pred, squared=False)

        if rmse < models_results[setname].bestourmethod_rmse:
            models_results[setname].bestourmethod_rmse = rmse
            models_results[setname].bestourmethod_rmse_name = method
            models_results[setname].y_pred_bestourmethod_rmse = y_pred

bestmnethodscount = {}
setofbestourmethodswtamd = {}

print("Results for inside and our methods")
print("%40s"% "Dataset", " , ", \
    "Best inside method RMSE", " , ", \
    "RMSE", " , ", \
    "Best inside method WTMAD2", " , ", \
    "WTMAD2", " , ", \
    "Best our method RMSE", " , ", \
    "RMSE", " , ", \
    "Best our method WTMAD2", " , ", \
    "WTMAD2")
for setname in fullsetnames:
    if models_results[setname].bestourmethod_rmse_name in bestmnethodscount:
        bestmnethodscount[models_results[setname].bestourmethod_rmse_name] += 1
    else:
        bestmnethodscount[models_results[setname].bestourmethod_rmse_name] = 1

    if models_results[setname].bestourmethod_wtmad_name != "":
        if models_results[setname].bestourmethod_wtmad_name in setofbestourmethodswtamd:
            setofbestourmethodswtamd[models_results[setname].bestourmethod_wtmad_name] += 1
        else:
            setofbestourmethodswtamd[models_results[setname].bestourmethod_wtmad_name] = 1
          
    print("%40s"%setname, " , ", \
        "%10s"%models_results[setname].bestinsidemethod_rmse_name , " , ",\
        "%7.3f"%models_results[setname].bestinsidemethod_rmse, " , ", \
        "%10s"%models_results[setname].bestinsidemethod_wtmad_name , " , ", \
        "%7.3f"%models_results[setname].bestinsidemethod_wtmad, " , ", \
        "%10s"%models_results[setname].bestourmethod_rmse_name , " , ", \
        "%7.3f"%models_results[setname].bestourmethod_rmse, " , ", \
        "%10s"%models_results[setname].bestourmethod_wtmad_name , " , ", \
        "%7.3f"%models_results[setname].bestourmethod_wtmad)

In [ ]:
print("RMSE")
for method in bestmnethodscount:
    print("Best our method ", method, " count: ", bestmnethodscount[method])

print()
print("WTMAD2")
for method in setofbestourmethodswtamd:
    print("Best our method ", method, " count: ", setofbestourmethodswtamd[method])


In [6]:
#filter and generate equations
basicfeattouse = ["Potential_Energy", \
                "Kinetic_Energy", \
                "FINAL_SINGLE_POINT_ENERGY", \
                "Dispersion_correction", \
                "E(C)", \
                "E(X)", \
                "Two_Electron_Energy", \
                "Nuclear_Repulsion", \
                "One_Electron_Energy"]

featuresvalues_perset = {}
for setname in fullsetnames:
    featuresvalues_perset [setname] = []
    for val in allvalues_perset[setname]:
        featuresvalues_perset[setname].append({})
        for k in val:
            if k.find("energydiff") != -1:
                torm = k.replace("energydiff", "")
                for f in val[k]:
                    tocheck = f.replace(torm, "")
                    if tocheck in basicfeattouse:
                        keytouse = f.replace("-", "_")
                        keytouse = keytouse.replace("(", "")
                        keytouse = keytouse.replace(")", "")
                        featuresvalues_perset[setname][-1][keytouse] = val[k][f]

# for debug purposes
#for val in featuresvalues_perset:
#    print("======= START =======")
#    print(val, len(featuresvalues_perset[val]))
#    pp.pprint(featuresvalues_perset[val])
#    print("=======  END  =======")


In [ ]:

equations = {"EC" :"EC" , \
            "EX" : "EX", \
            "FSPE" : "FINAL_SINGLE_POINT_ENERGY", \
            "DC" : "Dispersion_correction", \
            "PE" : "Potential_Energy", \
            "KE" : "Kinetic_Energy", \
            "OEE" : "One_Electron_Energy", \
            "TEE" : "Two_Electron_Energy", \
            "NR" : "Nuclear_Repulsion"}

eq_featuresvalues_perset = \
    commonutils.equation_parser_compiler(equations, functionals, basis_sets, basicfeattouse, \
                              featuresvalues_perset)

featuresvalues_perset = deepcopy(eq_featuresvalues_perset)

In [ ]:
functional_to_use = "PBE"
basissets_touse = ["MINIX", "SVP"]

labels = []
features = {}
supersetnameslist = list(supersetnames.keys())
for setname in featuresvalues_perset:
    if setname in supersetnameslist:
        print("Setname: ", setname)
        for entry in featuresvalues_perset[setname]:
            labels.append(supersetnameslist.index(setname))
            #print("Entry: ", entry)
            for featurename in entry:
                if featurename.startswith(functional_to_use+"_"):
                    for basisset in basissets_touse:
                        if featurename.find(basisset) != -1:
                            if featurename not in features:
                                features[featurename] = []
                            features[featurename].append(entry[featurename])


In [ ]:
for k in features:
    print(k, len(features[k]))

print("Labels: ", len(labels))

In [ ]:
# build a classification model using RF 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = []
for k in features:
    X.append(features[k])

X = np.array(X).T
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

accuracy_scores = []

for numoftrees in range(10, 200, 10):
    rf = RandomForestClassifier(n_estimators=numoftrees, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

# plot the accuracy scores
plt.figure(figsize=(10,6))
plt.plot(range(10, 200, 10), accuracy_scores, color='blue', linestyle='dashed', marker='o', markerfacecolor='red', markersize=10)
plt.title('Accuracy scores vs. number of trees')
plt.xlabel('Number of trees')
plt.ylabel('Accuracy score')
plt.show()

print("Best accuracy score: ", max(accuracy_scores), " with ", (accuracy_scores.index(max(accuracy_scores))+1)*10, " trees")


In [ ]:
bestnumoftree   = (accuracy_scores.index(max(accuracy_scores))+1)*10

accuracy_scores = []
for depth in range(1, 10):
    rf = RandomForestClassifier(n_estimators=bestnumoftree, max_depth=depth, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

# plot the accuracy scores
plt.figure(figsize=(10,6))
plt.plot(range(1, 10), accuracy_scores, color='blue', linestyle='dashed', marker='o', markerfacecolor='red', markersize=10)
plt.title('Accuracy scores vs. depth')
plt.xlabel('Depth')
plt.ylabel('Accuracy score')
plt.show()

print("Best accuracy score: ", max(accuracy_scores), " with ", (accuracy_scores.index(max(accuracy_scores))+1), " depth")

In [ ]:
rf = RandomForestClassifier(n_estimators=bestnumoftree,  \
                            random_state=42)
rf.fit(X, y)
# predict the labels and accuracy 
y_pred = rf.predict(X)
accuracy = accuracy_score(y, y_pred)
print("Accuracy: ", accuracy)

In [ ]:
# model tio use 
rf = RandomForestClassifier(n_estimators=bestnumoftree,  \
                            random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(" Test Accuracy: ", accuracy)
y_pred = rf.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print("Train Accuracy: ", accuracy)
y_pred = rf.predict(X)
accuracy = accuracy_score(y, y_pred)
print("Total Accuracy: ", accuracy)

In [ ]:
# use graphviz to visualize the tree
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz


for i in range(3):
    tree = rf.estimators_[i]
    dot_data = export_graphviz(tree,
                               feature_names=list(features.keys()),  
                               filled=True,  
                               max_depth=2, 
                               impurity=False, 
                               proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)